In [4]:
# 安装最新版tts-autolabel
os.system("pip install -U tts-autolabel -f https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html")
# os.system("pip install -U tts-autolabel -i https://mirrors.aliyun.com/pypi/simple")
from modelscope.tools import run_auto_label

import requests
import zipfile
import os
import IPython.display as ipd 

from modelscope.metainfo import Trainers
from modelscope.trainers import build_trainer
from modelscope.utils.audio.audio_utils import TtsTrainType
from modelscope.models.audio.tts import SambertHifigan
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

Looking in indexes: https://mirrors.aliyun.com/pypi/simple
Looking in links: https://modelscope.oss-cn-beijing.aliyuncs.com/releases/repo.html


DEPRECATION: pytorch-lightning 1.7.7 has a non-standard dependency specifier torch>=1.9.*. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [5]:
def download_file(url, save_path):
    """
    指定的Url下载文件,到指定的save_path;
    """
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)
    print(f"文件下载完成,存盘到{save_path}！")

In [6]:
# 下载例子男声
url = "https://isv-data.oss-cn-hangzhou.aliyuncs.com/ics/MaaS/TTS/download_files/test_male.zip"
save_path = "/mnt/workspace/test_male.zip"
download_file(url, save_path)

文件下载完成,存盘到/mnt/workspace/test_male.zip！


In [7]:
# 定义压缩文件的路径
zip_path = '/mnt/workspace/test_male.zip'
# 指定解压缩的目标路径
target_path = '/mnt/workspace/test_wavs/'  # 解压缩后的文件将会被放在这个目录下

with zipfile.ZipFile(zip_path ,'r') as zf:
    for file in zf.namelist():
        file_path = file.split('/')
        if file_path[0] == 'test_male' and  file_path[1] != "":
            zf.extract(file, target_path)
# 移动到target_path,删除空目录:            
os.system(f'mv {target_path}test_male/*.* {target_path}')
os.system(f'rm -d {target_path}test_male')        

0

In [2]:
# 新建TTS-AutoLabel工作文件夹output_training_data
os.system("mkdir /mnt/workspace/output_training_data")

input_wav = "./test_wavs/"
output_data = "./output_training_data/"

ret, report = run_auto_label(input_wav=input_wav, work_dir=output_data, resource_revision="v1.0.7")

mkdir: 无法创建目录 “/mnt/workspace/output_training_data”: 文件已存在
2023-11-09 15:54:34,625 - modelscope - INFO - Use user-specified model revision: v1.0.7
Downloading: 100%|██████████| 185/185 [00:00<00:00, 15.9kB/s]
Downloading: 100%|█████████▉| 706M/706M [00:03<00:00, 233MB/s] 
Downloading: 100%|██████████| 6.24k/6.24k [00:00<00:00, 988kB/s]
INFO:root:2023-11-09 15:54:44
INFO:root:TTS-AutoLabel version: 1.1.8
INFO:root:TTS-AutoLabel resource path: /mnt/workspace/.cache/modelscope/damo/speech_ptts_autolabel_16k/model
INFO:root:Target sampling rate: 16000
INFO:root:Input wav dir: /mnt/workspace/test_wavs
INFO:root:Output data dir: /mnt/workspace/output_training_data
INFO:root:wav_preprocess start...
INFO:root:---  new folder...  ---
INFO:root:---  OK  ---
100%|██████████| 20/20 [00:00<00:00, 289.99it/s]
INFO:root:[VAD] chunk recordings for training.
INFO:root:wav cut by vad start...
  0%|          | 0/20 [00:00<?, ?it/s]INFO:root:wav 01_000010_S0000 is no need to cut
INFO:root:wav 01_000006_S0

In [3]:
# 基于PTTS-basemodel微调

pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_16k'
# pretrained_model_id = 'damo/speech_personal_sambert-hifigan_nsf_tts_zh-cn_pretrain_24k'

dataset_id = "./output_training_data/"
pretrain_work_dir = "./pretrain_work_dir/"

os.system("mkdir /mnt/workspace/pretrain_work_dir")
        
# 训练信息，用于指定需要训练哪个或哪些模型，这里展示AM和Vocoder模型皆进行训练
# 目前支持训练：TtsTrainType.TRAIN_TYPE_SAMBERT, TtsTrainType.TRAIN_TYPE_VOC
# 训练SAMBERT会以模型最新step作为基础进行finetune
train_info = {
    TtsTrainType.TRAIN_TYPE_SAMBERT: {  # 配置训练AM（sambert）模型
        'train_steps': 202,               # 训练多少个step 
        'save_interval_steps': 200,       # 每训练多少个step保存一次checkpoint
        'log_interval': 10               # 每训练多少个step打印一次训练日志
    }
}

# 配置训练参数，指定数据集，临时工作目录和train_info
kwargs = dict(
    model=pretrained_model_id,                  # 指定要finetune的模型
    model_revision = "v1.0.6",
    work_dir=pretrain_work_dir,                 # 指定临时工作目录
    train_dataset=dataset_id,                   # 指定数据集id
    train_type=train_info                       # 指定要训练类型及参数
)

trainer = build_trainer(Trainers.speech_kantts_trainer,
                        default_args=kwargs)

trainer.train()

2023-11-09 15:55:46,801 - modelscope - INFO - Use user-specified model revision: v1.0.6
Downloading: 100%|██████████| 4.07k/4.07k [00:00<00:00, 450kB/s]
2023-11-09 15:55:48,318 - modelscope - INFO - Use user-specified model revision: v1.0.6
Downloading: 100%|██████████| 138k/138k [00:00<00:00, 42.0MB/s]
Downloading: 100%|██████████| 288k/288k [00:00<00:00, 15.2MB/s]
Downloading: 100%|██████████| 327k/327k [00:00<00:00, 109MB/s]
Downloading: 100%|██████████| 535k/535k [00:00<00:00, 20.9MB/s]
Downloading: 100%|██████████| 506/506 [00:00<00:00, 315kB/s]
Downloading: 100%|██████████| 1.69M/1.69M [00:00<00:00, 27.6MB/s]
Downloading: 100%|██████████| 452k/452k [00:00<00:00, 16.2MB/s]
Downloading: 100%|██████████| 998k/998k [00:00<00:00, 20.4MB/s]
Downloading: 100%|█████████▉| 866M/866M [00:03<00:00, 249MB/s] 
Downloading: 100%|██████████| 150M/150M [00:00<00:00, 249MB/s] 
Downloading: 100%|██████████| 3.82k/3.82k [00:00<00:00, 2.40MB/s]
Downloading: 100%|██████████| 2.49k/2.49k [00:00<00:00,

text.cc: festival_Text_init
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed


festival_initialize() called more than once
100%|██████████| 40/40 [00:00<00:00, 15525.83it/s]
INFO:root:TextScriptConvertor.process:
Save script to: ./pretrain_work_dir/data/Script.xml
INFO:root:TextScriptConvertor.process:
Save metafile to: ./pretrain_work_dir/data/raw_metafile.txt
INFO:root:[AudioProcessor] Initialize AudioProcessor.
INFO:root:[AudioProcessor] config params:
INFO:root:[AudioProcessor] wav_normalize: True
INFO:root:[AudioProcessor] trim_silence: True
INFO:root:[AudioProcessor] trim_silence_threshold_db: 60
INFO:root:[AudioProcessor] preemphasize: False
INFO:root:[AudioProcessor] sampling_rate: 16000
INFO:root:[AudioProcessor] hop_length: 200
INFO:root:[AudioProcessor] win_length: 1000
INFO:root:[AudioProcessor] n_fft: 2048
INFO:root:[AudioProcessor] n_mels: 80
INFO:root:[AudioProcessor] fmin: 0.0
INFO:root:[AudioProcessor] fmax: 8000.0
INFO:root:[AudioProcessor] phone_level_feature: True
INFO:root:[AudioProcessor] se_feature: True
INFO:root:[AudioProcessor] norm_type

In [5]:
model_dir = os.path.abspath("./pretrain_work_dir")

custom_infer_abs = {
    'voice_name':
    'F7',
    'am_ckpt':
    os.path.join(model_dir, 'tmp_am', 'ckpt'),
    'am_config':
    os.path.join(model_dir, 'tmp_am', 'config.yaml'),
    'voc_ckpt':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan', 'ckpt'),
    'voc_config':
    os.path.join(model_dir, 'orig_model', 'basemodel_16k', 'hifigan',
             'config.yaml'),
    'audio_config':
    os.path.join(model_dir, 'data', 'audio_config.yaml'),
    'se_file':
    os.path.join(model_dir, 'data', 'se', 'se.npy')
}
kwargs = {'custom_ckpt': custom_infer_abs}

model_id = SambertHifigan(os.path.join(model_dir, "orig_model"), **kwargs)



2023-11-09 16:00:10,614 - modelscope - INFO - am_config=/mnt/workspace/pretrain_work_dir/tmp_am/config.yaml voc_config=/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/config.yaml
2023-11-09 16:00:10,615 - modelscope - INFO - audio_config=/mnt/workspace/pretrain_work_dir/data/audio_config.yaml
2023-11-09 16:00:10,615 - modelscope - INFO - am_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400000.pth'), (2400200, '/mnt/workspace/pretrain_work_dir/tmp_am/ckpt/checkpoint_2400200.pth')])
2023-11-09 16:00:10,615 - modelscope - INFO - voc_ckpts=OrderedDict([(2400000, '/mnt/workspace/pretrain_work_dir/orig_model/basemodel_16k/hifigan/ckpt/checkpoint_2400000.pth')])
2023-11-09 16:00:10,616 - modelscope - INFO - se_path=/mnt/workspace/pretrain_work_dir/data/se/se.npy se_model_path=/mnt/workspace/pretrain_work_dir/orig_model/se.onnx
2023-11-09 16:00:10,616 - modelscope - INFO - mvn_path=/mnt/workspace/pretrain_work_dir/orig_model/mvn.npy
fe

Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Load pinyin_en_mix_dict failed
Removing weight norm...


In [26]:
# 将pretrain_work_dir 压缩成一个压缩包:
pretrain_dir = "./pretrain_work_dir"
pretrain_zip_path = "./pretrain_work_dir.zip"
with zipfile.ZipFile(pretrain_zip_path, 'w') as z:
    for root, dirs, files in os.walk(pretrain_dir):
        for file in files:
            z.write(os.path.join(root,file))  
         

In [ ]:
inference = pipeline(task=Tasks.text_to_speech, model=model_id)
output = inference(input="你还好吗?")

ipd.Audio(output["output_wav"], rate=16000)